## BIO-OPT_Inventory_Generator

This script generates 100 inventories (in this case, for vanillin production), where the percentage of woodchips allocated to vanillin production decreases from 100% to 1%. The remaining woodchips, not used for vanillin production, are assumed to be burned for steam generation. The amount of steam produced is calculated based on the calorific value (25.17 MJ/kg) and an 80% conversion efficiency.

### For each percentage of woodchips used for vanillin, the following are calculated:

1. Steam produced from the remaining woodchips: The amount of woodchips not used for vanillin production is converted into steam using the calorific value of woodchips and the specified efficiency.
2. Ash production: 8% of the remaining woodchips are converted into ash after the steam production process.
3. Vanillin production: The amount of vanillin produced is directly proportional to the percentage of woodchips used for its production.
4. Water usage, energy, and chemical inputs: These are scaled according to the percentage of woodchips used for vanillin production.

### Constants:
Total woodchips for 100% vanillin production: 0.306 kg<br>
Calorific value of woodchips: 25.17 MJ/kg<br>
Steam production efficiency: 80%<br>
Ash percentage: 8%<br>

### Outputs:
A CSV or Excel file containing the inventories with the following columns:

Woodchips for Vanillin (kg)<br>
Sodium Hydroxide (kg)<br>
Electricity (kWh)<br>
Steam (MJ)<br>
Water (L)<br>
CO2 as a Catalyst (kg)<br>
Acetic Acid (kg)<br>
Dichloromethane (kg)<br>
Secondary Steam (MJ)<br>
Vanillin Produced (kg)<br>
Wastewater Treatment (m3)<br>
Steam from Remaining Woodchips (MJ)<br>
Ash Produced (kg)<br>

The results are saved in an Excel file with 100 tabs, each corresponding to one of the inventories generated using decreased percentage of woodchips allocated for vanillin production.

### Requirements:
The following Python libraries and tools are essential for running the script:

1. `pandas`: Used for data manipulation and analysis, particularly for reading and writing data from various file formats. Install with: `pip install pandas==1.3.3`

2. `numpy`: Used for numerical operations and handling arrays. Install with: `pip install numpy==1.21.2`

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
# Constants
woodchips_total = 0.306  # Total woodchips for 100% vanillin production (kg)
calorific_value = 27.15  # MJ/kg for woodchips
efficiency = 0.80  # Conversion efficiency to steam
ash_percentage = 0.08  # 8% of remaining woodchips are turned into ash


# Other constants for inputs
sodium_hydroxide = 0.065178  # kg
electricity_autoclave = 0.6885  # kWh
steam_input = 0.823157  # MJ
water = 0.01836  # L
co2_catalyst = 0.0186354  # kg
acetic_acid = 0.000000193  # kg
dichloromethane = 0.0244188  # kg
vanillin_total = 0.026928  # kg
wastewater_treatment_total = 3.27857E-06  # m3

In [ ]:
# Initialize Excel writer
output_dir = "./output" #Set the output directory to a local 'output' folder in the same directory as the script.
output_path = os.path.join(output_dir, "Vanillin_LCI.xlsx")
writer = pd.ExcelWriter(output_path, engine='xlsxwriter')

In [ ]:
# Headers for each tab
headers = [
    "name", "amount", "unit", "database", "categories", "location", 
    "type", "group_tag", "uncertainty type", "loc", "scale", "shape", 
    "minimum", "maximum", "reference product"
]

In [ ]:
# Specification data for each input/output
specifications = {
    "market for electricity, medium voltage": {
        "unit": "kilowatt hour", "database": "ecoinvent 3.7 cutoff", "location": "CA-BC",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "electricity, medium voltage"
    },
    "market for heat, from steam, in chemical industry": {
        "unit": "Megajoule", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "heat, from steam, in chemical industry"
    },
    "market for heat, from steam, in chemical industry (production)": {
        "unit": "Megajoule", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "heat, from steam, in chemical industry"
    },
    "market for sodium hydroxide, without water, in 50% solution state": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "GLO",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "sodium hydroxide, without water, in 50% solution state"
    },
    "wood chips production, softwood, at sawmill": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "CA-QC",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "wood chips, wet, measured as dry mass"
    },
    "Vanillin production": {
        "unit": "kilogram", "database": "biorefinery", "location": "BC-CA",
        "type": "production", "group_tag": "main_product", "reference product": "Vanillin production"
    },
    "market for acetic acid, without water, in 98% solution state": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "GLO",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "acetic acid, without water, in 98% solution state"
    },
    "market for carbon dioxide, liquid": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "carbon dioxide, liquid"
    },
    "market for dichloromethane": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "dichloromethane"
    },
    "water production, deionised": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "manu_materials", "reference product": "water, deionised"
    },
    "market for wastewater, average": {
        "unit": "cubic meter", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "waste_treatment", "reference product": "wastewater, average"
    },
    "market for wood ash mixture, pure": {
        "unit": "kilogram", "database": "ecoinvent 3.7 cutoff", "location": "RoW",
        "type": "technosphere", "group_tag": "waste_treatment", "reference product": "wood ash mixture, pure"
    }
}

In [ ]:
# Loop from 100% to 1% of woodchips used for Vanillin production
for percentage in range(100, 0, -1):
    woodchips_for_vanillin = woodchips_total * (percentage / 100)  # Woodchips for vanillin
    woodchips_for_steam = woodchips_total - woodchips_for_vanillin  # Remaining woodchips for Steam

    # Calculate the amount of steam produced from remaining woodchips
    steam_from_woodchips = woodchips_for_steam * calorific_value * efficiency
    
    # Calculate the amount of ash produced from the remaining woodchips
    ash_from_woodchips = woodchips_for_steam * ash_percentage

    solid_waste = (woodchips_for_vanillin + (sodium_hydroxide * (percentage / 100)) + (co2_catalyst * (percentage / 100)) + (acetic_acid * (percentage / 100)) + (dichloromethane * (percentage / 100)) + (woodchips_for_steam)) - (vanillin_total * (percentage / 100))
    steam_output = (solid_waste * 27.15 * 0.8)# MJ
    ash = solid_waste * 0.08 #kg
    
    # Create an inventory dictionary for this percentage
    inventory = {
        # Inputs
        "market for electricity, medium voltage": electricity_autoclave * (percentage / 100),
        "market for heat, from steam, in chemical industry": steam_input * (percentage / 100),
        "market for sodium hydroxide, without water, in 50% solution state": sodium_hydroxide * (percentage / 100),
        "wood chips production, softwood, at sawmill": woodchips_for_vanillin,
        "market for acetic acid, without water, in 98% solution state": acetic_acid * (percentage / 100),
        "market for carbon dioxide, liquid": co2_catalyst * (percentage / 100),
        "market for dichloromethane": dichloromethane * (percentage / 100),
        "water production, deionised": water * (percentage / 100),

        # Outputs
        "Vanillin production": vanillin_total * (percentage / 100),
        "market for wastewater, average": wastewater_treatment_total * (percentage / 100),
        # The remaining woodchips are used to generate steam and ash, no "solid waste"
        "market for heat, from steam, in chemical industry (production)": -(steam_output),
        "market for wood ash mixture, pure": ash
    }
    

    # Create a DataFrame for the inventory and append the specifications for each flow
    rows = []
    for name, amount in inventory.items():
        spec = specifications.get(name, {})
        rows.append({
            "name": name,
            "amount": amount,
            "unit": spec.get("unit", ""),
            "database": spec.get("database", ""),
            "categories": None,
            "location": spec.get("location", ""),
            "type": spec.get("type", ""),
            "group_tag": spec.get("group_tag", ""),
            "uncertainty type": None,
            "loc": None,
            "scale": None,
            "shape": None,
            "minimum": None,
            "maximum": None,
            "reference product": spec.get("reference product", "")
        })

    df_inventories = pd.DataFrame(rows)
    

    # Write this inventory to a new sheet in the Excel file, named after the percentage
    sheet_name = f'{percentage}% woodchips'
    df_inventories.to_excel(writer, sheet_name=sheet_name, index=False, startrow=16, header=headers)

    # Add additional information to specific cells
    worksheet = writer.sheets[sheet_name]
    
    worksheet.write('A1', 'cutoff')
    worksheet.write('B1', 16)
    
    worksheet.write('A6', 'Activity', writer.book.add_format({'bold': True}))
    worksheet.write('B6', f'Vanillin {percentage}% production', writer.book.add_format({'bold': True}))

    worksheet.write('A7', 'categories')
    worksheet.write('B7', 'Chemical Manufacturing')
    
    worksheet.write('A8', 'code')
    worksheet.write('B8', f'ThisIsFU{percentage}')  # Incremental code based on percentage of woodchips

    worksheet.write('A9', 'comment')
    worksheet.write('B9', 'vanillin production as one of the outputs of a wood-based biorefinery')

    worksheet.write('A10', 'filename')
    worksheet.write('B10', 'just4thesake')

    worksheet.write('A11', 'location')
    worksheet.write('B11', 'BC-CA')

    worksheet.write('A12', 'production amount')
    worksheet.write('B12', 1)

    worksheet.write('A13', 'type')
    worksheet.write('B13', 'process')

    worksheet.write('A14', 'unit')
    worksheet.write('B14', 'kilogram')

    worksheet.write('A15', 'reference product')
    worksheet.write('B15', f'Vanillin {percentage}% production')

    worksheet.write('A16', 'Exchanges')
    
    worksheet.write_formula('A26', '=B15')
    worksheet.write_formula('O26', '=B15')
    
    
    # Set A28 to "market for heat, from steam, in chemical industry"
    worksheet.write('A28', 'market for heat, from steam, in chemical industry')
    worksheet.write_formula('D26',  "='[Vanillin.xlsx]100% woodchips'!B2") #Write a formula that references a cell from the '100% woodchips' tab in the 'Vanillin.xlsx' file,

    
    # Save the workbook with all the tabs
writer.save()
    #inventories.append(inventory)

In [ ]:
print(f"Excel file with 100 tabs saved to: {output_path}")